In [1]:
import os


In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\riswa\\Desktop\\Text-Summarizer-Project'

## entity ---return type of a function

In [33]:
from  dataclasses import  dataclass
from pathlib import Path

In [26]:
@dataclass(frozen =True)
class DataIngestionConfig:
    root_dir:Path
    source_URL :str
    local_data_file: Path
    unzip_dir : Path
    

In [27]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories,get_size

In [34]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
    

## update the components

In [35]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size
from pathlib import Path

In [38]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [41]:
try:
    config = ConfigurationManager(config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH)
    print("config")
    
    data_ingestion_config = config.get_data_ingestion_config()
    print('data_ingestion_config')
    data_ingestion = DataIngestion(config=data_ingestion_config)
    print("data_ingestion")
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-09 15:49:31,383: INFO: common: yaml file Loaded Successfullt]
[2024-07-09 15:49:31,391: INFO: common: yaml file Loaded Successfullt]


TypeError: Subscripted generics cannot be used with class and instance checks

In [45]:
from src.textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [47]:
from dataclasses import dataclass
from pathlib import Path
from typing import List
import os
import urllib.request as request
import zipfile
import yaml

# Utility functions
# def read_yaml(file_path: Path) -> dict:
#     with open(file_path, 'r') as file:
#         return yaml.safe_load(file)

# def create_directories(paths: List[Path]):
#     for path in paths:
#         os.makedirs(path, exist_ok=True)

# def get_size(path: Path) -> str:
#     size = os.path.getsize(path)
#     return f"{size} bytes"

# Data classes
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# ConfigurationManager class
class ConfigurationManager:
    def __init__(self, config_filepath: Path, params_filepath: Path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config['artifacts_root'])])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        create_directories([Path(config['root_dir'])])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config['root_dir']),
            source_URL=config['source_URL'],
            local_data_file=Path(config['local_data_file']),
            unzip_dir=Path(config['unzip_dir'])
        )
        return data_ingestion_config

# DataIngestion class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with headers:\n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"UNzippinf the data from {self.config.local_data_file}")
# # Paths to config files (example)
# CONFIG_FILE_PATH = Path("path/to/config.yaml")
# PARAMS_FILE_PATH = Path("path/to/params.yaml")

# Execution
try:
    config_manager = ConfigurationManager(config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH)
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-07-09 16:00:50,634: INFO: 3798969871: artifacts\data_ingestion\data.zip downloaded with headers:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5BC6:E4C9B:2ECCCA:3EB188:668D0F98
Accept-Ranges: bytes
Date: Tue, 09 Jul 2024 10:30:46 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10241-MAA
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1720521046.300680,VS0,VE277
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: fe3d42770dbe689857c15d5c5e677b4ceedda416
Expires: Tue, 09 Jul 2024 10:35:46 GMT
Source-Age: 0

]
[2024-07-09 16:00:51,048: INFO: 3798969871: UNzippin